## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,hoa binh,VN,2021-05-30 16:39:35,20.8133,105.3383,85.89,70,100,2.39,overcast clouds
1,1,pevek,RU,2021-05-30 16:39:35,69.7008,170.3133,37.89,70,0,5.64,clear sky
2,2,lebu,CL,2021-05-30 16:39:35,-37.6167,-73.6500,56.82,70,100,6.06,overcast clouds
3,3,tasiilaq,GL,2021-05-30 16:39:36,65.6145,-37.6368,39.36,80,99,17.54,overcast clouds
4,4,rikitea,PF,2021-05-30 16:39:36,-23.1203,-134.9692,74.68,79,100,16.31,light rain


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,hoa binh,VN,2021-05-30 16:39:35,20.8133,105.3383,85.89,70,100,2.39,overcast clouds
10,10,nuevitas,CU,2021-05-30 16:39:37,21.5453,-77.2644,85.44,55,19,4.29,few clouds
12,12,kieta,PG,2021-05-30 16:39:38,-6.2167,155.6333,78.03,85,77,4.27,broken clouds
13,13,bandarbeyla,SO,2021-05-30 16:39:38,9.4942,50.8122,79.54,85,22,29.86,few clouds
25,25,arraial do cabo,BR,2021-05-30 16:39:40,-22.9661,-42.0278,78.76,78,75,11.50,broken clouds
29,29,guasdualito,VE,2021-05-30 16:39:41,7.2424,-70.7324,86.00,70,59,5.75,broken clouds
30,30,puerto carreno,CO,2021-05-30 16:39:42,6.1890,-67.4859,81.01,80,66,6.98,broken clouds
34,34,puerto ayora,EC,2021-05-30 16:36:36,-0.7393,-90.3518,80.56,81,69,7.00,broken clouds
35,35,boyolangu,ID,2021-05-30 16:39:43,-8.1181,111.8935,75.40,93,91,3.80,overcast clouds
39,39,ode,NG,2021-05-30 16:39:44,7.7899,5.7117,79.63,76,76,5.12,moderate rain


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                209
City                   209
Country                209
Date                   209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                209
City                   209
Country                209
Date                   209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,hoa binh,VN,85.89,overcast clouds,20.8133,105.3383,
10,nuevitas,CU,85.44,few clouds,21.5453,-77.2644,
12,kieta,PG,78.03,broken clouds,-6.2167,155.6333,
13,bandarbeyla,SO,79.54,few clouds,9.4942,50.8122,
25,arraial do cabo,BR,78.76,broken clouds,-22.9661,-42.0278,
29,guasdualito,VE,86.00,broken clouds,7.2424,-70.7324,
30,puerto carreno,CO,81.01,broken clouds,6.1890,-67.4859,
34,puerto ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
35,boyolangu,ID,75.40,overcast clouds,-8.1181,111.8935,
39,ode,NG,79.63,moderate rain,7.7899,5.7117,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        r_hotels = requests.get(base_url, params= params).json()

        hotel_df.loc[index, 'Hotel Name'] = r_hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')
        
hotel_df.head(10)
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,hoa binh,VN,85.89,overcast clouds,20.8133,105.3383,Hoa Dao Hotel
10,nuevitas,CU,85.44,few clouds,21.5453,-77.2644,Hotel Caonaba
12,kieta,PG,78.03,broken clouds,-6.2167,155.6333,Uruna Bay Retreat
13,bandarbeyla,SO,79.54,few clouds,9.4942,50.8122,JABIR HOTEL
25,arraial do cabo,BR,78.76,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
29,guasdualito,VE,86.00,broken clouds,7.2424,-70.7324,Hotel Cuibas
30,puerto carreno,CO,81.01,broken clouds,6.1890,-67.4859,Casa Hotel Victoria
34,puerto ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
35,boyolangu,ID,75.40,overcast clouds,-8.1181,111.8935,Yos ega cv
39,ode,NG,79.63,moderate rain,7.7899,5.7117,MORALAB HOTEL


In [ ]:
# 7. Drop the rows where there is no Hotel Name.


In [30]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
hotel_df.to_csv(path_or_buf='Weather_Database/WeatherPy_vacation.csv', index_label='City_ID')

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# # 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))